# Install required libraries

In [1]:
!pip install scikit-image==0.19.3
!pip install filterpy
!pip install scikit-learn
!pip install ultralytics opencv-python-headless torch
!pip install ultralytics deep_sort_realtime
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 77.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-image: filename=scikit_image-0.19.3-cp311-cp311-linux_x86_64.whl size=33913333 sha256=97f516b8ffc6223a127b8064a2e2b6f0f7b4f68b2aac6aa25538e3c8c70fc10c
  Stored in directory: /root/.cache/pip/wheels/7b/12/cd/f311cabf9e8708d1e29e9951ee0839fb89b69e1acc60c94927
Successfully built scikit-image
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.25.2
    Uninstalling scikit-image-0.25.2:
      Successfully uninstalled scikit-image-0.25.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110460 sha256=2848c2acf92284772e4b1bc55501a495ab92fb56dda8a365cc300be96893d376
  Stored in

# Prerequisites to do before start
# (upload the 3 MARS files first from model_data directory in submission)

In [2]:
!mkdir model_data

!mv /content/mars-small128.ckpt-68577 /content/model_data
!mv /content/mars-small128.ckpt-68577.meta /content/model_data
!mv /content/mars-small128.pb /content/model_data

# Clone Deepsort repo from github

In [3]:
!git clone https://github.com/nwojke/deep_sort.git

Cloning into 'deep_sort'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 167 (delta 7), reused 7 (delta 7), pack-reused 147 (from 2)
Receiving objects: 100% (167/167), 84.64 KiB | 5.64 MiB/s, done.
Resolving deltas: 100% (87/87), done.


# Main Workflow

In [5]:
from ultralytics import YOLO
import cv2
import os
from tracker import Tracker

# Load model and video
model = YOLO("best.pt")
video_path = "/content/15sec_input_720p.mp4"
out_path = "/content/deepsort_custom.mp4"

cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

tracker = Tracker()
colors = [(225,5,0), (0,225,5), (5, 0, 225)]

while True:
    ret, frame = cap.read()
    if not ret:
        break

    detections = []
    results = model(frame)[0]

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        if conf > 0.6:
            detections.append([x1, y1, x2, y2, conf])

    tracker.update(frame, detections)

    for track in tracker.tracks:
        x1, y1, x2, y2 = map(int, track.bbox)
        cv2.rectangle(frame, (x1, y1), (x2, y2), colors[track.track_id % 3], 2)
        cv2.putText(frame, f"ID: {track.track_id}", (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, colors[track.track_id % 3], 2)

    out.write(frame)

cap.release()
out.release()
print("Tracking complete. Video saved to:", out_path)



0: 384x640 1 ball, 16 players, 2 referees, 3211.7ms
Speed: 17.1ms preprocess, 3211.7ms inference, 32.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 2 referees, 2307.2ms
Speed: 3.2ms preprocess, 2307.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 2256.0ms
Speed: 2.9ms preprocess, 2256.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 2257.9ms
Speed: 3.0ms preprocess, 2257.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 3236.5ms
Speed: 3.1ms preprocess, 3236.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 2259.4ms
Speed: 3.3ms preprocess, 2259.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 2249.1ms
Speed: 4.4ms preprocess, 2249.1ms inference, 1.0m